In [1]:
from collections import namedtuple
# neptune
import os
import numpy as np
from transformers import BertForSequenceClassification
from time import sleep
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.nn import functional as F
from models.t5 import *
from data.data import *
from data.prerpocessing import *
device="cuda"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gersa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
class Args:
    def __init__(self):
        self.data_path =  "D:\Github/decepticon/Processed_New"
        self.batch_size =  16
        self.num_workers =  0
        self.pretrained_model = "t5-small"
        self.pretrained_eval_model = "iarfmoose/bert-base-cased-qa-evaluator"
        self.tokenizer_len = 32102 ## manual
        self.padding_id = 0
        self.special_tokens = ["<answer>", "<context>"]
    def to_dict(self):
        out = dict()
        out["data_path"] = self.data_path
        out["batch_size"] = self.batch_size
        out["num_workers"] = self.num_workers
        out["pretrained_model"] = self.pretrained_model
        out["tokenizer_len"] = self.tokenizer_len
        out["padding_id"] = self.padding_id
        out["special_tokens"] = self.special_tokens
        return out

hparams=Args()

In [ ]:
model = T5FinetuneForRACE.load_from_checkpoint("D:/Github/decepticon/project/checkpoints/0_02.ckpt").to(device)
evaluator = BertForSequenceClassification.from_pretrained(hparams.pretrained_eval_model)
tokenizer = AutoTokenizer.from_pretrained("iarfmoose/bert-base-cased-qa-evaluator")

In [ ]:
data_module = RaceDataModule(hparams, custom_collate_fn = RaceDataModule.t5_collate_fn)
data_module.prepare_data()
data_module.setup()

In [ ]:
T5FinetuneForRACE.datamodule = data_module

In [ ]:
x, y = next(iter(data_module.val_dataloader()))
output = model.generate_questions(x["input_ids"].to(device), decode=True)


In [ ]:
#output = model.generate_questions(x["input_ids"].to(device), decode=True, num_beams=10, no_repeat_ngram_size=2, num_return_sequences = 5)
output = model.generate_top(x["input_ids"].to(device), decode=True, num_return_sequences = 5)
batch_i = 15
for i in range(5):
    question = output[batch_i][i]
    answer = model.datamodule.tokenizer.decode(x["input_ids"][batch_i]).replace("<answer>", "").lstrip().split("<context>")[0]
    i = tokenizer(text = question, 
              text_pair =  answer,  
              padding = True,
              max_length=512,
              truncation=True,
              return_tensors="pt")
    res = F.softmax(evaluator(i["input_ids"], attention_mask =i["attention_mask"])["logits"], dim=1)
    print("Question: %s\nAnswer: %s\nQuality: %.2f" %(question, answer, res.squeeze()[1].item()))
print("========================")
answer = model.datamodule.tokenizer.decode(x["input_ids"][batch_i]).replace("<answer>", "").lstrip().split("<context>")[0]
question = model.datamodule.tokenizer.decode(y["input_ids"][batch_i], skip_special_tokens=True, 
                                                            clean_up_tokenization_spaces=True)
i = tokenizer(text = question, 
              text_pair =  answer,  
              padding = True,
              max_length=512,
              truncation=True,
              return_tensors="pt")
res = F.softmax(evaluator(i["input_ids"], attention_mask =i["attention_mask"])["logits"], dim=1)
print("Original: %s\nAnswer: %s\nQuality: %.2f" %(question, answer, res.squeeze()[1].item()))


# ARCHIVE (Old codes)

In [8]:
"====================================================="

'====================================================='

In [50]:
question = output[9][0]
question = "Where is Frank going today?"

In [63]:
answer = model.datamodule.tokenizer.decode(x["input_ids"][2]).replace("<answer>", "").lstrip().split("<context>")[0]
answer = "To the hospital"

In [64]:
i = tokenizer(text = question, 
              text_pair =  answer,  
              padding = True,
              max_length=512,
              truncation=True,
              return_tensors="pt")


In [65]:
import torch.nn.functional as F

In [66]:
F.softmax(evaluator(i["input_ids"], attention_mask =i["attention_mask"])["logits"])

<ipython-input-66-5c1c71566942>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(evaluator(i["input_ids"], attention_mask =i["attention_mask"])["logits"])


tensor([[0.9639, 0.0361]], grad_fn=<SoftmaxBackward>)

In [8]:
def generate_question(content):
    model.eval()        
    generated = model.model.generate(input_ids = x["input_ids"].to(device), 
                                     num_beams = 6,
                                     num_return_sequences = 5,
                                     max_length = 50,
                                     no_repeat_ngram_size = 2,
                                     early_stopping = True)

    output = []
    for sequence in generated[:]:
        output.append(data_module.tokenizer.decode(sequence, skip_special_tokens= True, clean_up_tokenization_spaces = True))
    content = data_module.tokenizer.decode(content["input_ids"][0], skip_special_tokens= False, clean_up_tokenization_spaces = True)
    return output, content, generated

In [9]:
j = 0
for x, y in data_module.val_dataloader():
    output = generate_question(x)
    print("-----------------------------------------")
    print("GROUND TRUTH: %s" %data_module.tokenizer.decode(y["input_ids"][0], skip_special_tokens= True, clean_up_tokenization_spaces = True))
    for i, q in enumerate(output[0]):
        print("GENERATED %s: %s"  %(i, "".join(q)))
    print("ANSWER: %s" % "".join(output[1]).split("<context>")[0].replace("<answer>", "").lstrip())
    j += 1
    if j > 10: break

-----------------------------------------
GROUND TRUTH: The Sherman Antitrust Act _.
GENERATED 0: The Sherman Antitrust Act _.
GENERATED 1: According to the passage, the early tycoons _.
GENERATED 2: It can be inferred from the passage that the Sherman Antitrust Act _.
GENERATED 3: The first tycoons built successful businesses because they _.
GENERATED 4: The Sherman Antitrust Act was passed because it _.
GENERATED 5: According to the passage, tycoons _.
GENERATED 6: The early tycoons _.
GENERATED 7: According to the passage, monopolies _.
GENERATED 8: According to the passage, tycoons _
GENERATED 9: According to the passage, tycoons_.
GENERATED 10: Farmers and businesses _.
GENERATED 11: According to the passage, some farmers _.
GENERATED 12: Farmers were out of business because they _.
GENERATED 13: According to the passage, farmers _.
GENERATED 14: Some farmers _.
GENERATED 15: Foreign tourists are advised to learn some bargaining skills to _.
GENERATED 16: Foreign tourists should l

KeyboardInterrupt: 

In [20]:
output[2]

tensor([[   0,  363,  133,  ...,    0,    0,    0],
        [   0, 4073,   13,  ...,    0,    0,    0],
        [   0,  363,   31,  ...,    0,    0,    0],
        ...,
        [   0, 4073,   13,  ...,    1,    0,    0],
        [   0, 4073,   13,  ...,    1,    0,    0],
        [   0, 4073,   13,  ...,   58,    1,    0]], device='cuda:0')

In [23]:
x["input_ids"].shape 

torch.Size([16, 444])

In [24]:
output[2].shape

torch.Size([80, 17])

In [60]:
generated = model.model.generate(input_ids = x["input_ids"].to(device), 
                                     num_beams = 6,
                                     num_return_sequences = 5,
                                     max_length = 30,
                                     no_repeat_ngram_size = 2,
                                     early_stopping = True)

In [61]:
g = generated.view(16, 5, -1)

In [69]:
output=[]
for indx in range(16):    
    for sequence in g[indx]:
        output.append(data_module.tokenizer.decode(sequence, skip_special_tokens= True, clean_up_tokenization_spaces = True))


In [57]:
x["input_ids"].shape

torch.Size([16, 444])

In [84]:
my_list = data_module.tokenizer.batch_decode(generated, skip_special_tokens=True, clean_up_tokenization_spaces=True)
n=5
inal = [my_list[i * n:(i + 1) * n] for i in range((len(my_list) + n - 1) // n )] 

In [15]:
tokenizer(inal[0][0])

NameError: name 'inal' is not defined